In [1]:
import hyperspy.api as hys
import numpy as np
import matplotlib.pyplot as plt
import tkinter.filedialog as tkf

In [2]:
%matplotlib inline

In [ ]:
ll_adr = tkf.askopenfilename()
hl_adr = tkf.askopenfilename()
print(ll_adr)
print(hl_adr)

In [ ]:
ll = hys.load(ll_adr, signal_type="EELS")
hl = hys.load(hl_adr, signal_type="EELS")
print(ll)
print(hl)

In [ ]:
zlp_shift = ll.estimate_zero_loss_peak_centre().data
print(type(zlp_shift))
print(zlp_shift.shape)
spike = np.where(zlp_shift > 10)
print(spike)

In [ ]:
ll.align_zero_loss_peak(also_align=[hl], signal_range=(-10., 10.))
print(ll)
print(hl)

In [ ]:
ll_sum = np.sum(ll.data, axis=2)
print(ll_sum.shape)
print(np.max(ll_sum), np.min(ll_sum), np.mean(ll_sum))
hl_sum = np.sum(hl.data, axis=2)
print(hl_sum.shape)
print(np.max(hl_sum), np.min(hl_sum), np.mean(hl_sum))
total_sum = ll_sum + hl_sum
print(total_sum.shape)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8, 5))
ax[0].imshow(ll_sum)
ax[1].imshow(hl_sum)
ax[2].imshow(total_sum)
fig.tight_layout()
plt.show()

In [ ]:
hl_nor = hl.deepcopy()
hl_nor.data = hl_nor.data / total_sum[:, :, np.newaxis]
print(np.unique(np.sum(hl_nor, axis=2)))

In [ ]:
hl_nor.plot()

In [ ]:
# binning

cluster_size_y = 2
cluster_size_x = 2
stride_y = 2
stride_x = 2
hl_nor_binned = []
height = len(range(0, hl_nor.data.shape[0]-cluster_size_y+1, stride_y))
width = len(range(0, hl_nor.data.shape[1]-cluster_size_x+1, stride_x))
for i in range(0, hl_nor.data.shape[0]-cluster_size_y+1, stride_y):
    for j in range(0, hl_nor.data.shape[1]-cluster_size_x+1, stride_x):
        hl_nor_binned.append(np.sum(hl_nor.data[i:i+cluster_size_y, j:j+cluster_size_x], axis=(0, 1)))
        
hl_nor_binned = np.asarray(hl_nor_binned).reshape(height, width, -1)
hl_nor_binned = hys.signals.Signal1D(hl_nor_binned)
hl_nor_binned.metadata = hl_nor.metadata
hl_nor_binned.axes_manager = hl_nor.axes_manager
hl_nor_binned.axes_manager[0].size = width
hl_nor_binned.axes_manager[1].size = height
hl_nor_binned.axes_manager[0].scale = hl_nor.axes_manager[0].scale * cluster_size_x
hl_nor_binned.axes_manager[1].scale = hl_nor.axes_manager[1].scale * cluster_size_y
print(hl_nor_binned.axes_manager)

In [ ]:
hl_nor_binned.remove_background(zero_fill=True, fast=False)

In [ ]:
lok = 520.0
rok = 560.0
hl_ok = hl_nor_binned.isig[lok:rok]
print(hl_ok)

In [ ]:
hl_dm = hys.signals.Signal2D(np.rollaxis(np.rollaxis(hl_ok.data, 0, 3), 0, 3))
print(hl_dm)
hl_ok.save(hl_adr[:-4]+"_alined_integration_normalized_cropped_OK.hdf5")
hl_dm.save(hl_adr[:-4]+"_alined_integration_normalized_cropped_OK_dm_import.hdf5")

In [ ]:
ll_dm = hys.signals.Signal2D(np.rollaxis(np.rollaxis(ll.data, 0, 3), 0, 3))
print(ll_dm)
hl_dm = hys.signals.Signal2D(np.rollaxis(np.rollaxis(hl_nor.data, 0, 3), 0, 3))
print(hl_dm)

ll.save(ll_adr[:-4]+"_alined.hdf5")
ll_dm.save(ll_adr[:-4]+"_alined_dm_import.hdf5")
hl_nor.save(hl_adr[:-4]+"_alined_integration_normalized.hdf5")
hl_dm.save(hl_adr[:-4]+"_alined_integration_normalized_dm_import.hdf5")

In [ ]:
# crop the core loss spectra
l1 = 450.0
r1 = 480.0
hl_crop_1 = hl_nor.isig[l1:r1]
hl_crop_1.save(hl_adr[:-4]+"_integration_normalized_cropped_TiL23.hdf5")
hl_crop_1_dm = hys.signals.Signal2D(np.rollaxis(np.rollaxis(hl_crop_1.data, 0, 3), 0, 3))
hl_crop_1_dm.save(hl_adr[:-4]+"_integration_normalized_cropped_TiL23_dm_import.hdf5")

In [ ]:
# crop the core loss spectra
l2 = 520.0
r2 = 560.0
hl_crop_2 = hl_nor.isig[l2:r2]
hl_crop_2.save(hl_adr[:-4]+"_integration_normalized_cropped_OK.hdf5")
hl_crop_2_dm = hys.signals.Signal2D(np.rollaxis(np.rollaxis(hl_crop_2.data, 0, 3), 0, 3))
hl_crop_2_dm.save(hl_adr[:-4]+"_integration_normalized_cropped_OK_dm_import.hdf5")

In [ ]:
# crop the core loss spectra
l3 = 635.0
r3 = 670.0
hl_crop_3 = hl_nor.isig[l3:r3]
hl_crop_3.save(hl_adr[:-4]+"_cropped_MnL23.hdf5")
hl_crop_3_dm = hys.signals.Signal2D(np.rollaxis(np.rollaxis(hl_crop_3.data, 0, 3), 0, 3))
hl_crop_3_dm.save(hl_adr[:-4]+"_cropped_MnL23_dm_import.hdf5")